# Interfaces

Interfaces in Java allow common functionality (methods) to be declared. Several classes may implement an interface and a class may implement more than one interface. Contrast this with subclassing where in Java a class can only extend a single superclass. Interfaces would not be useful if not for the fact that it is possible to declare reference variables of a an Interface type. Similar to classes, interfaces belong to packages and are defined in a file named `<interface_name>.java`. The structure of an interface file is as follows:

```java
<package_declaration>
<imports>

<modifier> interface <interface_name> extends {

...

}
```

In contrast to classes, interfaces only declare methods, but typically do not provide an implementation (definition) for methods. Syntactically, a method declaration is the same as the definition except that the method body is ommitted. Since Java version 8 interfaces are allowed to implement instance (non-static) methods. Such methods are prefixed with the `default` keyword.

A class can implement zero or more interfaces. These are listed as part of the class declaration like this:

```java
public class MyClass implements interface1, interface2, ... {
    ...
}
```

Say we have some classes whose instances allow us to access their fields through a method `Object getFieldValue(String fieldName)`. Any class may implement this functionality indepenent of what superclass it is derived from. Thus, it may make sense to define this method in an interface.

In [2]:
package lecture;

public interface GenericFieldAccessor {
    
    // declare a method that every class implementing the interface must define
    public Object getFieldValue(String fieldName);
}

lecture.GenericFieldAccessor

Now let's define some classes that implement that interface.

In [14]:
package lecture;

public class Person implements GenericFieldAccessor {
    
    private String name;
    private int age;
  
    public Person (String name, int age) {
        this.name = name;
        this.age = age;
    }
    
    public Object getFieldValue(String fieldName) {
        if (fieldName.equals("name")) {
            return name;
        }
        else if (fieldName.equals("age")) {
            return new Integer(age);
        }
        // here we would throw an exception to indicate to the caller that the field name does not exist
        return null;
    }
    
}

lecture.Person

In [7]:
package lecture;

public class Car implements GenericFieldAccessor {
    
    private String brand;
    private String model;
    private int year;
    
    public Car (String brand, String model, int year) {
        this.brand = brand;
        this.model = model;
        this.year = year;
    }
    
    public Object getFieldValue(String fieldName) {
        if (fieldName.equals("brand")) {
            return brand;
        }
        if (fieldName.equals("model")) {
            return model;
        }
        if (fieldName.equals("year")) {
            return year;
        }
        return null;
    }
}

lecture.Car

Now we can call the `getFieldValue` method from any object of a class that implements the `GenericFieldAccessor` interface:

In [25]:
import lecture.Person;
import lecture.Car;
import lecture.GenericFieldAccessor;

GenericFieldAccessor x = new Person("Peter", 59);
GenericFieldAccessor y = new Car("Porsche", "311", 1979);

System.out.println(String.format("Person x has name %s and age %d", x.getFieldValue("name"), x.getFieldValue("age")));
System.out.println(String.format("Car y has brand %s and model %s", y.getFieldValue("brand"), y.getFieldValue("model")));

Person x has name Peter and age 59
Car y has brand Porsche and model 311


null

## Inheritance

In Java, an interface can be inherited from another interface. 

So far, `GenericFieldAccessor` is not that useful. First of a client still has to know exactly which fields a class provides. Futhermore, we still have to add this annoying boilerplate code to every class that implements the interface. We will solve the first problem now and come back to the second problem when we talk about **reflection**.


In [20]:
package lecture;

public interface GenericFieldAccessorPlus extends GenericFieldAccessor {
    
    public String[] getFieldNames();
}


lecture.GenericFieldAccessorPlus

In [21]:
package lecture;

public class PersonPlus implements GenericFieldAccessorPlus {
    
    private String name;
    private int age;

    public PersonPlus (String name, int age) {
        this.name = name;
        this.age = age;
    }
    
    public Object getFieldValue(String fieldName) {
        if (fieldName.equals("name")) {
            return name;
        }
        else if (fieldName.equals("age")) {
            return new Integer(age);
        }
        // here we would throw an exception to indicate to the caller that the field name does not exist
        return null;
    }
 
    public String[] getFieldNames() {
        return new String[]{ "name" , "age"};
    }
    
}

lecture.PersonPlus

In [24]:
import lecture.PersonPlus;
import lecture.GenericFieldAccessorPlus;

GenericFieldAccessorPlus x = new PersonPlus("Peter", 59);

for(String fieldName: x.getFieldNames()) {
    System.out.println(String.format("field <%s> has value <%s>", fieldName, x.getFieldValue(fieldName).toString()));
}


field <name> has value <Peter>
field <age> has value <59>


null